# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
# GitHub
# https://github.com/bluenets126/dsc-course0-m8-lab

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [2]:
#Load data into dataframe
aviation_df = pd.read_csv("./data/AviationData.csv", encoding='latin-1', low_memory=False)

aviation_df.head()


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [3]:
#Filter for only Airplane Category
aviation_df_cleaned = aviation_df[aviation_df['Aircraft.Category'] == 'Airplane']

#Filter for only non-Amateur Built
aviation_df_cleaned = aviation_df_cleaned[aviation_df_cleaned['Amateur.Built'] == 'No']

#Filter for incident dates within past 40 years (after 1985)
aviation_df_cleaned = aviation_df_cleaned[aviation_df_cleaned['Event.Date'] > '1985-01-01']

aviation_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21430 entries, 10688 to 88886
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                21430 non-null  object 
 1   Investigation.Type      21430 non-null  object 
 2   Accident.Number         21430 non-null  object 
 3   Event.Date              21430 non-null  object 
 4   Location                21424 non-null  object 
 5   Country                 21429 non-null  object 
 6   Latitude                19169 non-null  object 
 7   Longitude               19163 non-null  object 
 8   Airport.Code            13972 non-null  object 
 9   Airport.Name            14059 non-null  object 
 10  Injury.Severity         20617 non-null  object 
 11  Aircraft.damage         20205 non-null  object 
 12  Aircraft.Category       21430 non-null  object 
 13  Registration.Number     21224 non-null  object 
 14  Make                    21427 non-null 

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [4]:
#Calculate the total number of people on the plane and put as new column in dataframe (Total_Passengers)
aviation_df_cleaned['Total_Passengers'] = aviation_df_cleaned[['Total.Fatal.Injuries','Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured']].sum(axis=1)

#Remove Data with 0 total passengers
aviation_df_cleaned2 = aviation_df_cleaned.drop(aviation_df_cleaned[aviation_df_cleaned['Total_Passengers'] == 0].index)

#Total serious or fatal injuries
aviation_df_cleaned2['sf_injuries'] = aviation_df_cleaned2[['Total.Fatal.Injuries','Total.Serious.Injuries']].sum(axis=1)

#Ratio of injury per flight (serious/fatal)
aviation_df_cleaned['sf_injury_rate'] = aviation_df_cleaned2.apply(lambda x: x['sf_injuries']/x['Total_Passengers'], axis=1)



aviation_df_cleaned.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Total_Passengers,sf_injury_rate
10688,20001214X35509,Accident,DEN85LA064,1985-01-14,"WAPITI, WY",United States,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,VMC,Maneuvering,Probable Cause,12-01-2016,2.0,0.500000
11638,20001214X36510,Accident,LAX85LA257,1985-05-13,"MESA, AZ",United States,NaN,NaN,FFZ,FALCON FLD.,...,NaN,1.0,NaN,NaN,VMC,Approach,Probable Cause,03-08-2011,1.0,1.000000
11898,20001214X36887,Accident,NYC85FA145B,1985-06-11,"BELMAR, NJ",United States,NaN,NaN,BLM,BELMAR MONMOUTH CO.,...,1.0,1.0,4.0,NaN,VMC,Takeoff,Probable Cause,08-04-2013,6.0,0.333333
12384,20001214X37274,Accident,NYC85LA188,1985-07-21,"SIDNEY, ME",United States,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,VMC,Takeoff,Probable Cause,01-02-2016,4.0,0.000000
12683,20001214X37356,Incident,ATL85IA251,1985-08-16,"HILTON HEAD, SC",United States,NaN,NaN,49J,HILTON HEAD,...,NaN,NaN,NaN,4.0,VMC,Landing,Probable Cause,26-07-2011,4.0,0.000000


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [5]:
#Create column to say if aircraft was destroyed
#If Aircraft damage is destrayed Yes, if minor/substatial No, Else Unknown

#Initialization of new column Destoryed with default value Unknown
aviation_df_cleaned['Destroyed'] = 'Unknown'

#If aviation.damage column is Destoryed then Put Yes in Destroyed column
aviation_df_cleaned.loc[aviation_df_cleaned['Aircraft.damage'].str.strip() == 'Destroyed', 'Destroyed'] = 'Yes'

#If aviation.damage column is Minor or Substatial then put No in Destroyed column
aviation_df_cleaned.loc[aviation_df_cleaned['Aircraft.damage'].str.strip() == 'Substantial', 'Destroyed'] = 'No'
aviation_df_cleaned.loc[aviation_df_cleaned['Aircraft.damage'].str.strip() == 'Minor', 'Destroyed'] = 'No'



aviation_df_cleaned.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Total_Passengers,sf_injury_rate,Destroyed
10688,20001214X35509,Accident,DEN85LA064,1985-01-14,"WAPITI, WY",United States,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,VMC,Maneuvering,Probable Cause,12-01-2016,2.0,0.500000,Yes
11638,20001214X36510,Accident,LAX85LA257,1985-05-13,"MESA, AZ",United States,NaN,NaN,FFZ,FALCON FLD.,...,1.0,NaN,NaN,VMC,Approach,Probable Cause,03-08-2011,1.0,1.000000,Yes
11898,20001214X36887,Accident,NYC85FA145B,1985-06-11,"BELMAR, NJ",United States,NaN,NaN,BLM,BELMAR MONMOUTH CO.,...,1.0,4.0,NaN,VMC,Takeoff,Probable Cause,08-04-2013,6.0,0.333333,Yes
12384,20001214X37274,Accident,NYC85LA188,1985-07-21,"SIDNEY, ME",United States,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,VMC,Takeoff,Probable Cause,01-02-2016,4.0,0.000000,No
12683,20001214X37356,Incident,ATL85IA251,1985-08-16,"HILTON HEAD, SC",United States,NaN,NaN,49J,HILTON HEAD,...,NaN,NaN,4.0,VMC,Landing,Probable Cause,26-07-2011,4.0,0.000000,No


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [6]:
#Get count of distrinct Makes to check
#distinct_makes = aviation_df_cleaned['Make'].nunique()
print(aviation_df_cleaned['Make'].head())

aviation_df_cleaned['Make'] = aviation_df_cleaned['Make'].str.upper().str.strip()
aviation_df_cleaned['Model'] = aviation_df_cleaned['Model'].str.upper().str.strip()

#How many times each make appears - Saved as a DF and reset index?
aircraft_df = aviation_df_cleaned['Make'].value_counts().to_frame()
#print(aircraft_df)
aircraft_df.reset_index(inplace=True)
#aircraft_list2 = aircraft_df['Make'].to_list()
#print(aircraft_list2)


#Drop all rows where the make appears less than 30 times (48 remain)
aircraft_df = aircraft_df.drop(aircraft_df[aircraft_df['count'] < 30].index)
#print(len(aircraft_df))

    
#Create list of aircrafts to keep.  Will be used to pass through aviation_df_cleaned to delete rows
aircraft_list = aircraft_df['Make'].to_list()
#print(aircraft_list)

#Delete all rows from aviation_df_cleaned where the Make is not in the aircraft_list
aviation_df_cleaned = aviation_df_cleaned[aviation_df_cleaned['Make'].isin(aircraft_list)]

aviation_df_cleaned.info()
#print(aviation_df_cleaned.info())
#print(aviation_df_test.info())
#aircraft_list
#aviation_df_cleaned = aviation_df_cleaned[

10688    Cessna
11638    Cessna
11898    Cessna
12384      Lake
12683    Cessna
Name: Make, dtype: object
<class 'pandas.core.frame.DataFrame'>
Index: 18443 entries, 10688 to 88886
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                18443 non-null  object 
 1   Investigation.Type      18443 non-null  object 
 2   Accident.Number         18443 non-null  object 
 3   Event.Date              18443 non-null  object 
 4   Location                18438 non-null  object 
 5   Country                 18442 non-null  object 
 6   Latitude                16489 non-null  object 
 7   Longitude               16486 non-null  object 
 8   Airport.Code            12040 non-null  object 
 9   Airport.Name            12141 non-null  object 
 10  Injury.Severity         17715 non-null  object 
 11  Aircraft.damage         17345 non-null  object 
 12  Aircraft.Category       18443 non-null 

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [7]:
#aircraft_make_df = aircraft_appearance = aviation_df_cleaned['Make'].value_counts().to_frame()
#aircraft_model_df = aircraft_appearance = aviation_df_cleaned['Make'].value_counts().to_frame()

#print(len(aircraft_make_df))
#print(len(aircraft_model_df))

#Drop any row that doesn't have a model associated to it
aviation_df_cleaned = aviation_df_cleaned.dropna(subset=['Model'])

#Combine aircraft make and model to new column named MakeModel
aviation_df_cleaned['MakeModel'] = aviation_df_cleaned['Make'].str.upper().str.cat(aviation_df_cleaned['Model'], sep='-')






#aircraft_makemodel_df.head()
aviation_df_cleaned.head()




,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Total_Passengers,sf_injury_rate,Destroyed,MakeModel
10688,20001214X35509,Accident,DEN85LA064,1985-01-14,"WAPITI, WY",United States,NaN,NaN,NaN,NaN,...,1.0,NaN,VMC,Maneuvering,Probable Cause,12-01-2016,2.0,0.500000,Yes,CESSNA-182Q
11638,20001214X36510,Accident,LAX85LA257,1985-05-13,"MESA, AZ",United States,NaN,NaN,FFZ,FALCON FLD.,...,NaN,NaN,VMC,Approach,Probable Cause,03-08-2011,1.0,1.000000,Yes,CESSNA-T303
11898,20001214X36887,Accident,NYC85FA145B,1985-06-11,"BELMAR, NJ",United States,NaN,NaN,BLM,BELMAR MONMOUTH CO.,...,4.0,NaN,VMC,Takeoff,Probable Cause,08-04-2013,6.0,0.333333,Yes,CESSNA-152
12683,20001214X37356,Incident,ATL85IA251,1985-08-16,"HILTON HEAD, SC",United States,NaN,NaN,49J,HILTON HEAD,...,NaN,4.0,VMC,Landing,Probable Cause,26-07-2011,4.0,0.000000,No,CESSNA-441
13114,20001214X37718,Accident,ATL85MA286,1985-09-29,"JENKINSBURG, GA",United States,NaN,NaN,NaN,WEST WIND SPORT PARACHUTE,...,NaN,NaN,VMC,Takeoff,Probable Cause,17-10-2016,17.0,1.000000,Yes,CESSNA-208


### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [8]:
#Standardize Engine Type field
aviation_df_cleaned['Engine.Type'].value_counts()
aviation_df_cleaned['Engine.Type'] = aviation_df_cleaned['Engine.Type'].replace('Geared Turbofan', 'Turbo Fan').str.strip()
aviation_df_cleaned['Engine.Type'] = aviation_df_cleaned['Engine.Type'].replace('UNK', 'Unknown').str.strip()

#Standardize Weather field
aviation_df_cleaned['Weather.Condition'].value_counts()
aviation_df_cleaned['Weather.Condition'] = aviation_df_cleaned['Weather.Condition'].replace('UNK', 'Unknown').str.strip()
aviation_df_cleaned['Weather.Condition'] = aviation_df_cleaned['Weather.Condition'].replace('Unk', 'Unknown').str.strip()

#Standardize Purpose of Flight
aviation_df_cleaned['Purpose.of.flight'].value_counts()
aviation_df_cleaned['Purpose.of.flight'] = aviation_df_cleaned['Purpose.of.flight'].replace('Air Race/show', 'Air Race show').str.strip()
aviation_df_cleaned['Purpose.of.flight'] = aviation_df_cleaned['Purpose.of.flight'].replace('Executive/corporate', 'Business').str.strip()
aviation_df_cleaned['Purpose.of.flight'] = aviation_df_cleaned['Purpose.of.flight'].replace('Public Aircraft - Federal', 'Public Aircraft').str.strip()
aviation_df_cleaned['Purpose.of.flight'] = aviation_df_cleaned['Purpose.of.flight'].replace('Public Aircraft - State', 'Public Aircraft').str.strip()
aviation_df_cleaned['Purpose.of.flight'] = aviation_df_cleaned['Purpose.of.flight'].replace('Public Aircraft - Local', 'Public Aircraft').str.strip()

#Standardize broad phase of flight. Assuming Descent/Approach is part of Landing and Climb is part of takeoff
aviation_df_cleaned['Broad.phase.of.flight'].value_counts()
aviation_df_cleaned['Broad.phase.of.flight'] = aviation_df_cleaned['Broad.phase.of.flight'].replace('Descent', 'Landing').str.strip()
aviation_df_cleaned['Broad.phase.of.flight'] = aviation_df_cleaned['Broad.phase.of.flight'].replace('Approach', 'Landing').str.strip()
aviation_df_cleaned['Broad.phase.of.flight'] = aviation_df_cleaned['Broad.phase.of.flight'].replace('Climb', 'Takeoff').str.strip()

aviation_df_cleaned['Broad.phase.of.flight'].value_counts()



#Fill blank injuries with 0.  Making the assumption if empty that there were no injuries.
aviation_df_cleaned['Total.Fatal.Injuries'] = aviation_df_cleaned['Total.Fatal.Injuries'].fillna(0)
aviation_df_cleaned['Total.Serious.Injuries'] = aviation_df_cleaned['Total.Serious.Injuries'].fillna(0)
aviation_df_cleaned['Total.Minor.Injuries'] = aviation_df_cleaned['Total.Minor.Injuries'].fillna(0)
aviation_df_cleaned['Total.Uninjured'] = aviation_df_cleaned['Total.Uninjured'].fillna(0)








aviation_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18427 entries, 10688 to 88886
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                18427 non-null  object 
 1   Investigation.Type      18427 non-null  object 
 2   Accident.Number         18427 non-null  object 
 3   Event.Date              18427 non-null  object 
 4   Location                18422 non-null  object 
 5   Country                 18426 non-null  object 
 6   Latitude                16480 non-null  object 
 7   Longitude               16477 non-null  object 
 8   Airport.Code            12032 non-null  object 
 9   Airport.Name            12131 non-null  object 
 10  Injury.Severity         17701 non-null  object 
 11  Aircraft.damage         17330 non-null  object 
 12  Aircraft.Category       18427 non-null  object 
 13  Registration.Number     18252 non-null  object 
 14  Make                    18427 non-null 

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [9]:
aviation_df_cleaned.info()

aviation_df_final = aviation_df_cleaned.drop(columns=['Airport.Code','Airport.Name','Aircraft.Category','Amateur.Built','Registration.Number','Schedule'])

aviation_df_final.info()

aviation_df_cleaned['Amateur.Built'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Index: 18427 entries, 10688 to 88886
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                18427 non-null  object 
 1   Investigation.Type      18427 non-null  object 
 2   Accident.Number         18427 non-null  object 
 3   Event.Date              18427 non-null  object 
 4   Location                18422 non-null  object 
 5   Country                 18426 non-null  object 
 6   Latitude                16480 non-null  object 
 7   Longitude               16477 non-null  object 
 8   Airport.Code            12032 non-null  object 
 9   Airport.Name            12131 non-null  object 
 10  Injury.Severity         17701 non-null  object 
 11  Aircraft.damage         17330 non-null  object 
 12  Aircraft.Category       18427 non-null  object 
 13  Registration.Number     18252 non-null  object 
 14  Make                    18427 non-null 

Amateur.Built
No    18427
Name: count, dtype: int64

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [10]:
aviation_df_final.to_csv("./data/AviationData_Cleaned.csv", index=False)